# SASSIE Shipboard Meteorology Data 
<p>The Meteorology sensors mounted on the R/V Woldstad took atmospheric measurements measurements throughout the campaign. Meteorology data are downloaded and plotted using the code below! <b>Please run the 'Data Download and Metadata Viewing' and 'Supporting Code' sections in order before running the 'Figure Making Code'.</b> </p>

<p>The aim of this notebook is to assist the end user in exploratory data analysis by downloading the SASSIE data from NASA's PODAAC, opening the dataset and displaying it's associated metadata, and creating a few visualizations. This notebook was created by Elizabeth Westbrook. For questions and trouble shooting, please email westbrooke@uncw.edu.</p>


# Credential Entry
In this section, you shoud enter your EarthData username and password. <b> DO NOT enter usernames and passwords here which are sensitive.</b> If you do not already have an EarthData account, you can create one <a href="https://urs.earthdata.nasa.gov/">here</a> . </p>
    <p>Please enter your EarthData credentials below: </p>

In [ ]:
#Enter Earthdata Credentials: 
username = 'your_usename'
password = 'your_password'

# Data Download and Metadata Viewing
The code in this section will download the dataset from PO.DAAC and open it as an xarray object for metadata and variable attribute viewing.

In [ ]:
import numpy as np
import xarray as xr
import glob 
from datetime import datetime, timedelta 
import matplotlib.pyplot as plt
import cartopy
import matplotlib
import os
import sys
import pandas as pd
import requests
import cmasher as cmr

<h2>Download The Meteorology data from PO.DAAC</h2>
<p> All data from the SASSIE campaign is stored on NASA's PO.DAAC. The code in this section of the notebook will download SASSIE Meteorology data from PO.DAAC, which is accessed through EarthData. 

In [ ]:
##DIRECTORY TO SAVE MET DATA
dir_in = 'Data/MET/'

The next block of code creates the directory specified above and downloads the shipboard MET file to your binder session if it has not already been downloaded.<b> To download the dataset to your local disk from here, right click on the file you want to download and click 'download'. </b>

In [ ]:
#if the file has not already been downloaded, it is downloaded here:     
if not os.path.isfile(dir_in+'SASSIE_Fall_2022_shipboard_MET.nc'):
    os.makedirs(dir_in)
    url = 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SASSIE_L2_SHIPBOARD_METEOROLOGY_V1/SASSIE_Fall_2022_shipboard_MET.nc'
    with requests.Session() as session:
            session.auth = (username, password)
            r1 = session.request('get', url)
            r = session.get(r1.url, auth=(username, password))
            if r.status_code == 200:
                if r.ok:
                    with open(dir_in+'SASSIE_Fall_2022_shipboard_MET.nc', 'wb') as f:
                        f.write(r.content) 
                        print('Saving Input File: ' + dir_in+'SASSIE_Fall_2022_shipboard_MET.nc') 
            else:
                print("Error:", r.status_code)
                if r.status_code == 401:
                    print ('Your Username and/or password are incorrect. Please try again')
else: 
    print('Shipboard MET file is already in binder directory')

<h2> View The Metadata Inside the TSG File</h2>

The netCDF file has global metadata attributes and attributes associated with each variable. This next block will load data and metadata of the netCDF file into an xarray object (ds). <br> The data set will then be displayed in a clickable HTML format. 

In [ ]:
file = dir_in + '/SASSIE_Fall_2022_shipboard_MET.nc'
ds = xr.open_dataset(file)
ds

# Supporting Code 
The code in this section provides a set up for the figure making code below by defining directories for data and figures and creating functions that will be called to actually map the data

<h3>Create a Directory to Save Figures</h3>

The next block creates a directory for figures to be saved

In [ ]:
#LOCAL DIRECTORY TO SAVE FIGURES
fig_dir = 'Figures/MET/'
#FIGURE DIR 
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

<h3>Define a Colormap and Label for Each Variable in the File</h3>

Within this collection of jupyter notebooks, the colormaps used for each variable are held as consistant as possible across all datasets. This function defines the colormap and a label for the variable of interest. 

In [ ]:
#DEFINES COLORMAPS AND LABELS OF EACH VARIABLE IN THIS DATA SET
def define_variable_attributes(var): 
    if var == 'downwelling_longwave':
        var_label = 'Longwave Radiation (W/m$^{2}$)'
        colormap = 'PiYG'
    if var == 'shortwave_radiation': 
        colormap = 'BrBG'
        var_label = 'Shortwave Radiation (W/m$^{2}$)'
    if var == 'bulk_ustar': 
        colormap = 'PRGn'
        var_label = 'Friction Velocity (m/s)'
    if var == 'bulk_wind_stress': 
        colormap = 'PuOr'
        var_label = 'Wind Stress (N/m$^{2}$)'
    if var == 'bulk_upwelling_IR':
        var_label = 'Upwelling Longwave Radiation (W/m$^{2}$)'
        colormap = 'hot'
    if var == 'bulk_ocean_temp':
        var_label = 'Sea Surface Temperature ($^{\circ}$C)'
        colormap = 'plasma'
    if var == 'air_temperature_merged':
        var_label = 'Air Temperature ($^{\circ}$C)'
        colormap = 'magma'
    if var == 'air_pressure_setra':
        var_label = 'Air Pressure from SETRA(hPa)'
        colormap = 'Blues'
    if var == 'bulk_sensible_heat_flux':
        var_label = 'Sensible Heat Flux (W/m$^{2}$)'
        colormap = 'autumn'
    if var == 'bulk_latent_heat_flux':
        var_label = 'Latent Heat Flux (W/m$^{2}$)'
        colormap = 'cool'
    if var == 'bulk_latent_heat_flux_low':
        var_label = 'Latent Heat Flux Low Estimate (W/m$^{2}$)'
        colormap = 'cool'
    if var == 'bulk_latent_heat_flux_high':
        var_label = 'Latent Heat Flux High Estimate (W/m$^{2}$)'
        colormap = 'cool'
    if var == 'bulk_heatflux_high':
        var_label = 'Net Heat Flux High Estimate (W/m$^{2}$)'
        colormap = 'Wistia'
    if var == 'bulk_heatflux_low':
        var_label = 'Net Heat Flux Low Estimate (W/m$^{2}$)'
        colormap = 'Wistia'
    if var == 'bulk_heatflux':
        var_label = 'NetHeat Flux (W/m$^{2}$)'
        colormap = 'Wistia'
    if var == 'bulk_buoyancy_flux':
        var_label = 'Atmospheric Buoyancy Flux (W/m$^{2}$)'
        colormap = 'twilight'
    if var == 'bulk_drag_coeff':
        var_label = 'Wind Stress Transfer (Drag Coefficient)'
        colormap = 'reds' 
    if var == 'uw_bar_gill':
        var_label = 'Direct Covariance Stress(m$^{2}$/s$^{2}$)'
        colormap = 'reds'
    if var == 'bulk_U10':
        var_label = 'Wind Speed Adjusted to 10m (m/s)'
        colormap = 'cividis' 
    if var == 'bulk_U10_neutral':
        var_label = 'Neutral Wind Speed Adjusted to 10m (m/s)'
        colormap = 'cividis'
    if var == 'wspd_true_gill':
        var_label = 'True Wind Speed from Gill (m/s)'
        colormap = 'cividis'
    if var == 'wspd_rel_gill':
        var_label = 'Relative Wind Speed from Gill (m/s)'
        colormap = 'cividis'
    if var == 'wspd_true_metek':
        var_label = 'True Wind Speed from Metek (m/s)'
        colormap = 'cividis'
    if var == 'wspd_average':
        var_label = 'True Wind Speed Average (m/s)'
        colormap = 'cividis'
    if var == 'wspd_rel_metek':
        var_label = 'Relative Wind Speed from Metek (m/s)'
        colormap = 'cividis'
    if var == 'wdir_true_gill':
        var_label = 'True Wind Direction from North (Degree) Gill'
        colormap = 'twilight' 
    if var == 'wdir_rel_gill':
        var_label = 'Relative Wind Direction (Degree) Gill'
        colormap = 'twilight' 
    if var == 'wdir_true_metek':
        var_label = 'True Wind Direction from North (Degree) Metek'
        colormap = 'twilight'
    if var == 'wdir_rel_metek':
        var_label = 'Relative Wind Direction (Degree) Metek'
        colormap = 'twilight'
    if var == 'wdir_average':
        var_label = 'Average Wind Direction from North (Degree)'
        colormap = 'twilight'    
    return colormap,var_label

<h3>Are you exploring this entire data set, or just the data collected durring a specific time period?</h3>

<p> In order to efficiently accomplish the data collection goals of the SASSIE campaign, the cruise was broken up into five destinct plays. 
    <br>
    <br>Play 1 - Ice and Open Water Survey
    <br>Play 2 - Zig-Zag Open Water Survey 
    <br>Play 3 - Ice Survey
    <br>Play 4 - Drifter-Following Survey Boxes
    <br>Play 5 - Repeated Ice to Open Water Transect</p>
    <p>The following block of code identifies the start and end datetimes of each of these plays, seperates the dataset into subsets for each play, and puts the start dates, end dates, and subseted data into lists for easier acess later. </p>

In [ ]:
##Identify the start and end times of each play
play1_start = datetime(2022,9,8,8) 
play1_end = datetime(2022,9,13)
play2_start = datetime(2022,9,14)
play2_end = datetime(2022,9,16)
play3_start = datetime(2022,9,16)
play3_end = datetime(2022,9,18)
play4_start = datetime(2022,9,19)
play4_end  = datetime(2022,9,23)
play5_start = datetime(2022,9,25)
play5_end = datetime(2022,9,29)

#subset the dataset into each play. 
play1_data =ds.sel(time=slice(play1_start,play1_end))
play2_data =ds.sel(time=slice(play2_start,play2_end))
play3_data =ds.sel(time=slice(play3_start,play3_end))
play4_data =ds.sel(time=slice(play4_start,play4_end))
play5_data =ds.sel(time=slice(play5_start,play5_end))
campaign_data = ds.sel(time=slice(play1_start,play5_end))

#Create lists of start dates, end dates, and subsetted datasets. 
play_start_dates = [play1_start,play2_start,play3_start,play4_start,play5_start,play1_start]
play_end_dates = [play1_end,play2_end,play3_end,play4_end,play5_end,play5_end]
play_data_all = [play1_data,play2_data,play3_data,play4_data,play5_data,campaign_data]


<h3>Define a Function to Create a Map of the Study Area</h3>

The following function creates a map of the SASSIE study area, which is defined by minimum and maximum lat/lon values. These ranges can be changed later when the function is called to zoom in/out on the study area.

In [ ]:
def map_study_area(latmin = 70, latmax =74,lonmin=-157,lonmax=-140):
    global fig 
    global ax
    
    #create the map as a figure, set the lat and lon ranges, and add land + river data:
    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(projection=cartopy.crs.NorthPolarStereo(central_longitude=-150))
    ax.set_extent([lonmin,lonmax,latmin,latmax], crs=cartopy.crs.PlateCarree())
    ax.coastlines(color='k')  
    ax.add_feature(cartopy.feature.LAND, facecolor = '0.50',zorder=1)
    ax.add_feature(cartopy.feature.RIVERS,facecolor='blue')
    #Add lat and lon gridlines and labels:
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.3) #draw_labels=True gives lat labels.
    gl.ylocator = matplotlib.ticker.FixedLocator(np.arange(60,75,1))
    gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(-170,-140,2))
    gl.top_labels = False; gl.bottom_labels = True; gl.right_labels = False
    
    #Add markers for reference cities on the coast:
    if (latmin<71.2906) & (lonmin<-156.7886):
        utqiagvik = ax.scatter(-156.7886,71.2906,s=100,transform=cartopy.crs.PlateCarree(),c='red',marker = '*',label='Utqiagvik, AK',zorder=2)
    if (latmin<70.2002) & (lonmax>-148.4597):
        deadhorse = ax.scatter(-148.4597,70.2002,s=100,c='cyan',transform=cartopy.crs.PlateCarree(),marker = '*',label='Deadhorse, AK',zorder=2)

<h3>Configure Supporting Data to Add to Maps</h3>

<p>The functions for viewing and plotting this data set below have options to include bathymetry and/or AMSR ice data to add context to maps. If you are using these options, run
    <br>the following code blocks to:
    <br>1. Create a directory for AMSR ice data and acess bathymetry data from NOAA
    <br>2. Define functions that add these data to your map when called.</p>

<h4>Create Directory for Ice Data</h4>

In [ ]:
##LOCAL DIRECTORY TO SAVE AMSR SEA ICE DATA 
ice_dir = 'Ice_Data/'

#ICE DATA DIR
if not os.path.exists(ice_dir):
    os.makedirs(ice_dir)

<h4>Access NOAA Bathymetry Data</h4>

In [ ]:
##READ IN TOPOGRAPHY/BATHYMETRY DATA
url = 'http://ferret.pmel.noaa.gov/thredds/dodsC/data/PMEL/etopo2.nc'
etopodata = xr.open_dataset(url) 

<h4>Define a Function to Add Ice Data to a Map</h4>
This function will download the AMSR ice product at the start and end date of the time period specified and add it to the plot. 
<br>Ice data from the start date will be shown in blue and ice data from the end date will be shown in red. 

In [ ]:
def add_ice_data(start_date,end_date):
    
        #get ice data at start of time range
        play_start = start_date
        filename_si='AMSR_U2_L3_SeaIce12km_B04_'+str(play_start.year)+str(play_start.month).zfill(2)+str(play_start.day).zfill(2)+'.he5' 
        i=0
        time_tmp=play_start - timedelta(days=i)
        while os.path.isfile(ice_dir+filename_si)==False and i<15:
            time_tmp=play_start - timedelta(days=i)
            filename_si='AMSR_U2_L3_SeaIce12km_B04_'+str(time_tmp.year)+str(time_tmp.month).zfill(2)+str(time_tmp.day).zfill(2)+'.he5'
            url = 'https://n5eil01u.ecs.nsidc.org/AMSA/AU_SI12.001/'+str(time_tmp.year)+'.'+str(time_tmp.month).zfill(2)+'.'+str(time_tmp.day).zfill(2)+'/'+filename_si
            with requests.Session() as session:
                session.auth = (username, password)
                r = session.get(session.request('get', url).url, auth=(username, password))
                if (r.status_code == 200) & (r.ok):
                        with open(ice_dir+filename_si, 'wb') as f:
                            f.write(r.content) 
                            print('Saving Input File: ' + ice_dir+filename_si) 
                else:
                    print("Error:", r.status_code)
                    if r.status_code == 401:
                        print ('Your Username and/or password are incorrect. Please try again')
            i=i+1
            

        #get ice data at end of time range 
        play_end = end_date
        filename_si='AMSR_U2_L3_SeaIce12km_B04_'+str(play_end.year)+str(play_end.month).zfill(2)+str(play_end.day).zfill(2)+'.he5' 
        i=0
        time_tmp=play_end - timedelta(days=i)
        while os.path.isfile(ice_dir+filename_si)==False and i<15:
            time_tmp=play_end - timedelta(days=i)
            filename_si='AMSR_U2_L3_SeaIce12km_B04_'+str(time_tmp.year)+str(time_tmp.month).zfill(2)+str(time_tmp.day).zfill(2)+'.he5'
            url = 'https://n5eil01u.ecs.nsidc.org/AMSA/AU_SI12.001/'+str(time_tmp.year)+'.'+str(time_tmp.month).zfill(2)+'.'+str(time_tmp.day).zfill(2)+'/'+filename_si
            with requests.Session() as session:
                session.auth = (username, password)
                r = session.get(session.request('get', url).url, auth=(username, password))
                if (r.status_code == 200) & (r.ok):
                        with open(ice_dir+filename_si, 'wb') as f:
                            f.write(r.content) 
                            print('Saving Input File: ' + ice_dir+filename_si) 
                else:
                    print("Error:", r.status_code)
                    if r.status_code == 401:
                        print ('Your Username and/or password are incorrect. Please try again')       
            i=i+1

            
        #apply ice data at the end of the time range (ice sheet growth) in red
        filename_si='AMSR_U2_L3_SeaIce12km_B04_'+str(play_end.year)+str(play_end.month).zfill(2)+str(play_end.day).zfill(2)+'.he5' 
        if os.path.isfile(ice_dir+filename_si):
            dsc = xr.open_dataset(ice_dir+filename_si,group='HDFEOS/GRIDS/NpPolarGrid12km')
            dsd = xr.open_dataset(ice_dir+filename_si,group='HDFEOS/GRIDS/NpPolarGrid12km/Data Fields') 
            var1 = np.array(dsd.SI_12km_NH_ICECON_DAY.squeeze().values)
            indx = np.where((var1==120) | (var1==0)); var1[indx] = 0
            ice_growth_location = np.ma.masked_where(var1==0,var1)
            pp = ax.pcolormesh(dsc.lon,dsc.lat,ice_growth_location, 
                            vmin=0,vmax=100, # Set max and min values for plotting
                            cmap='Reds_r', shading='auto',   # shading='auto' to avoid warning
                            transform=cartopy.crs.PlateCarree())
            
        #apply ice data at the start of the time range in blue
        filename_si='AMSR_U2_L3_SeaIce12km_B04_'+str(play_start.year)+str(play_start.month).zfill(2)+str(play_start.day).zfill(2)+'.he5' 
        if os.path.isfile(ice_dir+filename_si):
            dsc = xr.open_dataset(ice_dir+filename_si,group='HDFEOS/GRIDS/NpPolarGrid12km')
            dsd = xr.open_dataset(ice_dir+filename_si,group='HDFEOS/GRIDS/NpPolarGrid12km/Data Fields') 
            var1 = np.array(dsd.SI_12km_NH_ICECON_DAY.squeeze().values)
            indx = np.where((var1==120) | (var1==0)); var1[indx] = 0
            ice_location = np.ma.masked_where(var1==0,var1)
            pp = ax.pcolormesh(dsc.lon,dsc.lat,ice_location, 
                            vmin=0,vmax=100, # Set max and min values for plotting
                            cmap='Blues_r', shading='auto',   # shading='auto' to avoid warning
                            transform=cartopy.crs.PlateCarree())
        

<h4>Define a Function to Index Relevant Bathymetry Data and Add it to the Map</h4>
This function will index bathymetry data from NOAA within the appropriate spatial range and add it to the map. 


In [ ]:
def add_bathy_data(latmin,latmax,lonmin,lonmax):
        topoin = etopodata.rose.values[0:-1:5,1:-1:5]
        lons = etopodata.etopo2_x.values[0:-1:5]
        lats = etopodata.etopo2_y.values[0:-1:5]
        lons_in_range = lons[np.where((lons >lonmin-1) & (lons<lonmax+1))]
        lats_in_range = lats[np.where((lats >latmin-1) & (lats<latmax+1))]
        topo_in_range = np.squeeze(topoin[np.squeeze(np.where((lats >latmin-1) & (lats<latmax+1))),:][:,np.where((lons >lonmin-1) & (lons<lonmax+1))])
        [bathy_lon,bathy_lat] = np.meshgrid(lons_in_range,lats_in_range)
        
        bathy = ax.contour(bathy_lon,bathy_lat,topo_in_range,np.arange(-6000,-1000,300),transform=cartopy.crs.PlateCarree(),cmap='gray',alpha = 0.2,zorder = 0)

# Figure Making Code 

<h2>Make Maps of This Data</h2>
<p>Using the code in this section, the user can plot the MET data on a map colored by time or some collected meteorlogical data variable with or without quiver arrows for wind speed and direction. The code blocks in the 'supporting code' section should be run first. </p>

<h3> Mapping the Time and Location of Meteorology Data Collection</h3>

<p>The map_ship_track function shows the track of the R/V Woldstad on a map, colored by time. Various features of this function: </p>
    <p>1. The user has the option to specify a specific play to show on the map.The default play shown is '6', which gives time and location data from the entire campaign.
    <br>2. This function adds bathymetry contours showing the position of the continental shelf by default for reference. 
    <br>3. The user has the option to add the AMSR ice product at the start and end of the time period shown by inputting ice_data=True. </p>

In [ ]:
##Map The Ship Track and contour by time 
def map_ship_track(play,bathymetry_data=True,ice_data=False):
    
    ##LOAD DATA FOR SELECTED PLAY
    time_data = np.squeeze(play_data_all[play-1]['time'])
    lat_data = np.squeeze(play_data_all[play-1]['latitude'])
    lon_data = np.squeeze(play_data_all[play-1]['longitude'])
    
    ##CREATE A MAP WITH LAND, CITY MARKERS, BATHYMETRY DATA, AND AMSR ICE DATA AT START AND END OF PLAY.
    #To show the whole campaign area, use latmin = 70, latmax =74,lonmin=-157,lonmax=-140
    latmin = min(lat_data.values)-1; latmax = max(lat_data.values)+1; lonmin = min(lon_data.values)-1; lonmax = max(lon_data.values)+1
    if play == 6: 
        latmin = 70
    
    #make the map
    map_study_area(latmin = latmin,lonmin=lonmin,latmax=latmax,lonmax=lonmax)
    
    #give it a title
    if play == 6:
        ax.set_title('Location of Meteorology Observations \nfrom Entire Campaign',fontsize=18) 
    else:
        ax.set_title('Location of Meteorology Observations \nfrom Play '+str(play),fontsize=18) 
        
    ##OPTIONAL MAP ADD-ONs
    if bathymetry_data == True:    
        add_bathy_data(latmin,latmax,lonmin,lonmax)
    if ice_data ==True:
        add_ice_data(play_start_dates[play-1],play_end_dates[play-1])
        
    ##APPLY SHIP DATA TO THE MAP 
    track = ax.scatter(lon_data, 
                 lat_data,s=1, 
                 c=time_data,cmap=cmr.neon,
                   transform=cartopy.crs.PlateCarree())
    cbar = fig.colorbar(track, ax=ax, orientation="horizontal", pad=0.1)
    cbar.set_label(label='Date',size='large',weight='bold')
    if play == 3:
        number_of_ticks = 3
    else:
        number_of_ticks = 5
    cbar_tick_array=(np.linspace(min(time_data.values).astype('int64'),max(time_data.values).astype('int64'),number_of_ticks))
    cbar.set_ticks(cbar_tick_array)
    cbar.set_ticklabels(pd.to_datetime(cbar_tick_array).date)
    
    ##SAVE FIGURE
    if not os.path.exists(fig_dir+'shiptrack'):
        os.makedirs(fig_dir+'shiptrack')
    if play==6:
        print('Saving Output Image:  '+fig_dir+'shiptrack/'+'SASSIE_SHIPTRACK_MET_entire_campaign.png')
        plt.savefig(fig_dir+'shiptrack/'+'SASSIE_SHIPTRACK_MET_entire_campaign.png',dpi='figure',format='png')
    else:
        print('Saving Output Image:  '+fig_dir+'shiptrack/'+'SASSIE_SHIPTRACK_MET_play'+'_'+str(play)+'.png')
        plt.savefig(fig_dir+'shiptrack/'+'SASSIE_SHIPTRACK_MET_play'+'_'+str(play)+'.png',dpi='figure',format='png')

In [ ]:
map_ship_track(6)

<h3>Mapping Atmospheric Data Along the Ship Track</h3>

<p>The map_met_data function shows the track of the R/V Woldstad on a map, colored by an atmospheric variable of your choosing from the file. Various features of this function: </p>
    <p>1. The user has the option to specify a specific play to show on the map.The default play shown is '6', which gives time and location data from the entire campaign.
    <br>2. This function adds bathymetry contours showing the position of the continental shelf by default for reference. 
    <br>3. The user has the option to add the AMSR ice product at the start and end of the time period shown by inputting ice_data=True.
    <br>4. The user must specify the variable to be plotted, as well as the minimum and maximum value for that variable.</p>

In [ ]:
## PLOT TEMP AND SALINITY DATA PLAY BY PLAY## (Version 2 with ice before and after)
def map_met_data(play,var,var_min,var_max,bathymetry_data=True,ice_data=False,shiptrack=True):

    #LOAD DATA FOR SELECTED PLAY
    np_var = np.squeeze(play_data_all[play-1][var])
    np_lat = np.squeeze(play_data_all[play-1]['latitude'])
    np_lon = np.squeeze(play_data_all[play-1]['longitude'])
    
    play_start = play_start_dates[play-1]
    play_end = play_end_dates[play-1]
    
    
    colormap,var_label = define_variable_attributes(var)
    
    ##CREATE A MAP WITH LAND, CITY MARKERS, BATHYMETRY DATA(optional), AND AMSR ICE DATA AT START AND END OF PLAY(optional).
    #To show the whole campaign area, use latmin = 70, latmax =74,lonmin=-157,lonmax=-140
    latmin = min(np_lat.values)-1
    latmax = max(np_lat.values)+1
    lonmin = min(np_lon.values)-1
    lonmax = max(np_lon.values)+1
    
    #make the map
    map_study_area(latmin=latmin,latmax=latmax,lonmin=lonmin,lonmax=lonmax)
    
    #give it a title
    if play == 6:
        ax.set_title('Shipboard '+ var_label+' Reading from Entire Campaign',fontsize=18) 
    else:
        ax.set_title('Shipboard '+var_label+' Reading from Play '+str(play),fontsize=18) 
        
  
    ##ADD OPTIONAL BATHYMETRY AND/OR ICE DATA
    
    if bathymetry_data==True: 
        add_bathy_data(latmin,latmax,lonmin,lonmax)
        
    if ice_data ==True:
        add_ice_data(play_start_dates[play-1],play_end_dates[play-1])
        
    if shiptrack == True:
        ##ADD SHIP TRACK DATA
        track = ax.plot(np_lon, 
                     np_lat,linewidth = 0.5,
                     c='black',
                       transform=cartopy.crs.PlateCarree(),label = 'Ship Track',zorder=1)
    plt.legend(loc=2)

    ##APPLY SHIPBOARD MET DATA
    track = ax.scatter(np_lon, 
                 np_lat,s=5, 
                 c=np_var,cmap=colormap,
                   transform=cartopy.crs.PlateCarree(),
                   vmin = var_min,
                   vmax = var_max)
                        
    matplotlib.pyplot.colorbar(track, ax=ax, orientation="horizontal", pad=0.1).set_label(label=var_label,size='large',weight='bold')
    
    ##SAVE FIGURE
    if not os.path.exists(fig_dir+var):
        os.makedirs(fig_dir+var)
    if play == 6:
        print('Saving Output Image:  '+fig_dir+var+'/'+'SASSIE_SHIPBOARD_MET_entire_campaign.png')
        plt.savefig(fig_dir+var+'/'+'SASSIE_SHIPBOARD_MET_entire_campaign.png',dpi='figure',format='png')
    else: 
        print('Saving Output Image:  '+fig_dir+var+'/'+'SASSIE_SHIPBOARD_MET_play'+'_'+str(play)+'.png')
        plt.savefig(fig_dir+var+'/'+'SASSIE_SHIPBOARD_MET_play'+'_'+str(play)+'.png',dpi='figure',format='png')
    

In [ ]:
map_met_data(4,'air_temperature_merged',-2,4)
map_met_data(3,'air_temperature_merged',-3,0.5,ice_data=True)

<h3>Mapping Atmospheric Data Along the Ship Track with wind arrows</h3>

<p>The plot_wind_data function shows the track of the R/V Woldstad on a map, colored by an atmospheric variable of your choosing from the file with quiver arrows showing the speed and direction of the wind. Various features of this function: </p>
    <p>1. The user has the option to specify a specific play to show on the map.The default play shown is '6', which gives time and location data from the entire campaign.
    <br>2. This function adds bathymetry contours showing the position of the continental shelf by default for reference. 
    <br>3. The user has the option to add the AMSR ice product at the start and end of the time period shown by inputting ice_data=True.
    <br>4. The user must specify the variable to be plotted, as well as the minimum and maximum value for that variable.</p>

In [ ]:
##PLOT WIND ARROW PLOT 
def plot_wind_data(play,var,varmin,varmax,bathymetry_data=True,ice_data=False,shiptrack=False):
    
    #LOAD DATA FOR SELECTED PLAY
    np_wdir = np.squeeze(play_data_all[play-1]['wdir_average'])
    np_wspd = np.squeeze(play_data_all[play-1]['wspd_average'])
    np_lat = np.squeeze(play_data_all[play-1]['latitude'])
    np_lon = np.squeeze(play_data_all[play-1]['longitude'])
    np_var = np.squeeze(play_data_all[play-1][var])
    np_wdir_adjusted = -np_wdir -90

    colormap,var_label = define_variable_attributes(var)
    
    play_start = play_start_dates[play-1]
    play_end = play_end_dates[play-1]
    
    
    ##CREATE A MAP WITH LAND, CITY MARKERS, BATHYMETRY DATA(optional), AND AMSR ICE DATA AT START AND END OF PLAY(optional).
    #To show the whole campaign area, use latmin = 70, latmax =74,lonmin=-157,lonmax=-140
    latmin = min(np_lat.values)-1; latmax = max(np_lat.values)+0.6; lonmin = min(np_lon.values)-1; lonmax = max(np_lon.values)+1
            
    #make a map 
    map_study_area(latmin=latmin,latmax=latmax,lonmin=lonmin,lonmax=lonmax)
    
    #give it a title
    if play == 6:
        ax.set_title('Shipboard Wind and ' +var_label+' Data \n Entire Campaign',fontsize=18) 
    else:
        ax.set_title('Shipboard Wind  and '+var_label+' Data \n Play '+str(play),fontsize=18) 
        
    ##ADD OPTIONAL BATHYMETRY AND/OR ICE DATA
    if bathymetry_data==True: 
            add_bathy_data(latmin,latmax,lonmin,lonmax)    
    if ice_data ==True:
        add_ice_data(play_start,play_end) 
    if shiptrack == True:
        ##ADD SHIP TRACK DATA
        track = ax.plot(np_lon, 
                     np_lat,linewidth = 0.5,
                     c='black',
                       transform=cartopy.crs.PlateCarree(),label = 'Ship Track',zorder=1)
    plt.legend(loc=2)

    ##APPLY WIND DATA
    u = np_wspd * np.cos(np.radians(np_wdir_adjusted))
    v = np_wspd * np.sin(np.radians(np_wdir_adjusted))
                      
    q = ax.quiver(np_lon.values,np_lat.values, u, v,np_var,scale = 400,width = 0.002,cmap = colormap,clim=[varmin,varmax],transform=cartopy.crs.PlateCarree())
    
    plt.colorbar(q, ax=ax, orientation="vertical", pad=0.1).set_label(label=var_label,size='large',weight='bold')
    #ADD REFERENCE ARROW
    lat_ref = latmin + abs(latmax-latmin)/8
    lon_ref = lonmax - abs(lonmax-lonmin)/4
    angle_ref = [-45+90]
    speed_ref = [10]
    
    u_ref = speed_ref * np.cos(np.radians(angle_ref))
    v_ref = speed_ref * np.sin(np.radians(angle_ref))
    
    q_ref = ax.quiver(np.array([lon_ref]),np.array([lat_ref]),np.array([u_ref]),np.array([v_ref]),scale = 400,width = 0.002,transform=cartopy.crs.PlateCarree())
    text = 'Reference Arrow \n10 m/s \n45 Degrees Clockwise of North'
    plt.text(lon_ref,lat_ref-0.15,text,transform=cartopy.crs.PlateCarree(),va='center',ha='center',multialignment='center')
    
    ##SAVE FIGURE
    if not os.path.exists(fig_dir+'quiver_plots/'+var):
        os.makedirs(fig_dir+'quiver_plots/'+var)
    if play == 6:
        print('Saving Output Image:  '+fig_dir+'quiver_plots/'+var+'/'+'SASSIE_SHIPBOARD_MET_entire_campaign.png')
        plt.savefig(fig_dir+'quiver_plots/'+var+'/'+'SASSIE_SHIPBOARD_MET_entire_campaign.png',dpi='figure',format='png')
    else: 
        print('Saving Output Image:  '+fig_dir+'quiver_plots/'+var+'/'+'SASSIE_SHIPBOARD_MET_play'+'_'+str(play)+'.png')
        plt.savefig(fig_dir+'quiver_plots/'+var+'/'+'SASSIE_SHIPBOARD_MET_play'+'_'+str(play)+'.png',dpi='figure',format='png')

In [ ]:
plot_wind_data(6,'air_temperature_merged',-6,4,shiptrack=True)

<h2>Make Timeseries Plots on a 2D Axis</h2>
The functions in this section plots atmospheric weather variables and atmospheric flux variables collected from the R/V Woldstad Meteorology probe durring a selected play on panneled time series figures.

#### Plot Wind speed, wind direction, and air temp

In [ ]:
def plot_shipboard_met_wind_timeseries(play): 
    
    #SECLECT AND LOAD PLAY DATA
    play_start = play_start_dates[play-1]
    play_end = play_end_dates[play-1]
    
    time_data = np.squeeze(play_data_all[play-1]['time'])
    wspd_data = np.squeeze(play_data_all[play-1]['wspd_average'])
    wdir_data = np.squeeze(play_data_all[play-1]['wdir_average'])
    air_temp_data = np.squeeze(play_data_all[play-1]['air_temperature_merged'])

    
    #MAKE A THREE PANNELED TIME SERIES PLOT
    fig,axs = plt.subplots(3,sharex=True)
    
    axs[0].plot(time_data,wspd_data,linewidth=0.4,color='k')
    axs[0].set_ylabel('True Wind\n Speed (m/s)',fontsize=9)
    axs[1].plot(time_data,wdir_data,linewidth=0.4,color='k')
    axs[1].set_ylabel('True Wind Direction\n(Degrees from North)',fontsize=9)
    axs[2].plot(time_data,air_temp_data,linewidth=0.4,color='k')
    axs[2].set_ylabel('Air \nTemerature ($^{\circ}$C)',fontsize=9)
    axs[2].set_xlabel('Date')
    
    plt.xticks(pd.date_range(start=time_data.values[0],end=time_data.values[-1],periods=3))
    
    
    if play <=5:
        fig.suptitle('Play '+str(play)+' Wind Speed, Wind Direction, and Air Temperature \nData from Shipboard Meteorology')
    if play == 6: 
        fig.suptitle('Wind Speed, Wind Direction, and Air Temperature\n Data from Shipboard Meteorology (Entire Campaign)')
        
    ##SAVE FIGURE 
    if not os.path.exists(fig_dir+'timeseries'):
        os.makedirs(fig_dir+'timeseries')
    if play==6: 
        print('Saving Output Image:  '+fig_dir+'timeseries/SASSIE_SHIPBOARD_Met_entire_campaign.png')
        plt.savefig(fig_dir+'timeseries/SASSIE_SHIPBOARD_Met_entire_campaign.png',dpi='figure',format='png')
    else:
        print('Saving Output Image:  '+fig_dir+'timeseries/SASSIE_SHIPBOARD_Met_play'+'_'+str(play)+'.png')
        plt.savefig(fig_dir+'timeseries/SASSIE_SHIPBOARD_Met_play'+'_'+str(play)+'.png',dpi='figure',format='png')
    

In [ ]:
plot_shipboard_met_wind_timeseries(6)

#### Plot atmospheric fluxes

In [ ]:
def plot_shipboard_met_flux_timeseries(play): 
    
    #SECLECT AND LOAD PLAY DATA
    play_start = play_start_dates[play-1]
    play_end = play_end_dates[play-1]
    
    time_data = np.squeeze(play_data_all[play-1]['time'])
    wind_stress_data = np.squeeze(play_data_all[play-1]['bulk_wind_stress'])
    sens_heat_data = np.squeeze(play_data_all[play-1]['bulk_sensible_heat_flux'])
    latent_heat_data = np.squeeze(play_data_all[play-1]['bulk_latent_heat_flux'])
    latent_heat_data_high = np.squeeze(play_data_all[play-1]['bulk_latent_heat_flux_high'])
    latent_heat_data_low = np.squeeze(play_data_all[play-1]['bulk_latent_heat_flux_low'])
    lw_rad_data = np.squeeze(play_data_all[play-1]['downwelling_longwave'])
    sw_rad_data = np.squeeze(play_data_all[play-1]['shortwave_radiation'])
    net_heat_data = np.squeeze(play_data_all[play-1]['bulk_heatflux'])
    net_heat_data_high = np.squeeze(play_data_all[play-1]['bulk_heatflux_high'])
    net_heat_data_low = np.squeeze(play_data_all[play-1]['bulk_heatflux_low'])


    
    #MAKE A THREE PANNELED TIME SERIES PLOT
    fig,axs = plt.subplots(4,sharex=True,figsize=(10,10))
    
    axs[0].plot(time_data,wind_stress_data,linewidth=0.4,color='k')
    axs[0].set_ylabel('Wind Stress \n(Pa)',fontsize=14)

    
    axs[1].plot(time_data,sens_heat_data,linewidth=0.6,color='blue',label = 'Sensible Heat Flux')
    axs[1].plot(time_data,latent_heat_data,linewidth=0.6,color='red',label = 'Latent Heat Flux')
    axs[1].plot(time_data,latent_heat_data_high,linewidth=0.1,color='red',linestyle = '--')
    axs[1].plot(time_data,latent_heat_data_low,linewidth=0.1,color='red',linestyle = '--')
    axs[1].set_ylabel('Heat Fluxes\n (W/m$^2$)',fontsize=14)
    axs[1].plot(time_data,np.zeros(len(time_data)),linewidth=0.4,color='k')
    axs[1].set(xlim = [min(time_data),max(time_data)])
    axs[1].legend(loc=1)

    axs[2].plot(time_data,lw_rad_data,linewidth=0.4,color='cyan',label='Downwelling Longwave')
    axs[2].plot(time_data,sw_rad_data,linewidth=0.4,color='magenta',label='Shortwave')
    axs[2].set_ylabel('Radiation Fluxes \n (W/m$^2$)',fontsize=14)
    axs[2].legend(loc=1)
    
    axs[3].plot(time_data,net_heat_data,linewidth=1.2,color='tab:purple')
    axs[3].plot(time_data,net_heat_data_high,linewidth=0.4,color='tab:purple',linestyle = '--')
    axs[3].plot(time_data,net_heat_data_low,linewidth=0.4,color='tab:purple',linestyle='--')
    axs[3].set_ylabel('Net Heat Flux \n (W/m$^2$)',fontsize=14)
    axs[3].set_xlabel('Date',fontsize=14)
    axs[3].plot(time_data,np.zeros(len(time_data)),linewidth=0.4,color='k')
    
    fig.text(0.145,0.655,'Positive = Out of the Ocean',fontsize=12)
    fig.text(0.145,0.455,'Positive = Into the Ocean',fontsize=12)
    fig.text(0.145,0.255,'Positive = Out of the Ocean',fontsize=12)


    plt.xticks(pd.date_range(start=time_data.values[0],end=time_data.values[-1],periods=4))
    
    if play <=5:
        fig.suptitle('Play '+str(play)+' Air Pressure, Flux, and Water Temperature \nData from Shipboard Meteorology',y=0.94,fontsize=16)
    if play == 6: 
        fig.suptitle('Air Pressure, Flux, and Water Temperature\n Data from Shipboard Meteorology (Entire Campaign)',y=0.94,fontsize=16)
        
    ##SAVE FIGURE 
    if not os.path.exists(fig_dir+'timeseries'):
        os.makedirs(fig_dir+'timeseries')
    if play==6: 
        print('Saving Output Image:  '+fig_dir+'timeseries/SASSIE_SHIPBOARD_Fluxes_entire_campaign.png')
        plt.savefig(fig_dir+'timeseries/SASSIE_SHIPBOARD_Fluxes_entire_campaign.png',dpi='figure',format='png')
    else:
        print('Saving Output Image:  '+fig_dir+'timeseries/SASSIE_SHIPBOARD_Fluxes_play'+'_'+str(play)+'.png')
        plt.savefig(fig_dir+'timeseries/SASSIE_SHIPBOARD_Fluxes_play'+'_'+str(play)+'.png',dpi='figure',format='png')
    

In [ ]:
plot_shipboard_met_flux_timeseries(6)